In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import cv2
import time
from skimage import morphology
import seaborn as sns
import collections
plt.rcParams['figure.figsize'] = (20,12)
%config InlineBackend.figure_format = 'retina'


In [17]:
def main1(file_name, threshold_P, threshold_O, threshold_F):
    F_on_mito = []
    F_out_mito = []
    F_on_mito_list = []
    F_out_mito_list = []
    r_F_on_mito = []
    r_F_on_mito_list = []
    F_sum = []
    F_sum_list = []

    for i in range(1,4):
        PDZD8 =  cv2.imread(file_name + str(i) + ".tif_PDZD8.tif",cv2.IMREAD_ANYDEPTH)
        Mito = cv2.imread(file_name + str(i) + ".tif_OxPHOS.tif",cv2.IMREAD_ANYDEPTH)
        FKBP8 = cv2.imread(file_name + str(i) + ".tif_FKBP8.tif",cv2.IMREAD_ANYDEPTH)
        
        ret_M ,th_M = cv2.threshold(Mito,threshold_O,65535,cv2.THRESH_BINARY)
        th_M = th_M/65535

        ret_F ,th_F = cv2.threshold(FKBP8,threshold_F,65535,cv2.THRESH_BINARY)
        th_F = th_F/65535
        FKBP8 = FKBP8 - threshold_F
        FKBP8 = FKBP8 * th_F
    
        ret_P ,th_P = cv2.threshold(PDZD8,threshold_P,65535,cv2.THRESH_BINARY)
        th_P = th_P/65535
        PDZD8 = PDZD8 - threshold_P
        PDZD8 = PDZD8 * th_P
    
        # Converting to 8bit image
        th_P = (th_P).astype('uint8')
        th_F = (th_F).astype('uint8')
        th_M = (th_M).astype('uint8')

        #Outside of Mito
        th_M_conv = np.zeros_like(th_M)
        th_M_conv = np.where(th_M == 1, 0, 1)
        th_M_conv = (th_M_conv).astype('uint8')
   
        # FKBP8 on Mito
        th_F_on_M = th_F * th_M
        F_on_mito = np.sum(FKBP8 * th_F_on_M)
        #FKBP8 outside of Mito
        th_F_out_M = th_F * th_M_conv
        F_out_mito = np.sum(FKBP8 * th_F_out_M)
        
        # r_FKBP8 on Mito
        rotated_F = cv2.rotate(th_F, cv2.ROTATE_180)
        rotated_FKBP8 = cv2.rotate(FKBP8, cv2.ROTATE_180)
        th_r_F_on_M = rotated_F * th_M
        r_F_on_mito = np.sum(rotated_FKBP8 * th_r_F_on_M)
    
        F_on_mito_list = np.append(F_on_mito_list,F_on_mito)
        F_out_mito_list = np.append(F_out_mito_list,F_out_mito)
        r_F_on_mito_list = np.append(r_F_on_mito_list,r_F_on_mito)
        F_sum = np.sum(FKBP8)
        F_sum_list = np.append(F_sum_list,F_sum)
        
    #Mander's coefficient
    
    FKBP8_on_mito = np.sum(F_on_mito_list)/np.sum(F_sum_list)
    r_FKBP8_on_mito = np.sum(r_F_on_mito_list)/np.sum(F_sum_list)
    FKBP8_out_mito = np.sum(F_out_mito_list)/np.sum(F_sum_list)
    
    ans = [FKBP8_on_mito,r_FKBP8_on_mito, FKBP8_out_mito]
    return ans

In [18]:
df1_1 = pd.DataFrame(data=None, index=None, columns= ["FKBP8_on_mito","r_FKBP8_on_mito","FKBP8_out_mito"], dtype=None, copy=False)
for i in [1,6,7,9,10]:
    file_name = r"/Users/sa/Desktop/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 " + str(i) + " - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    threshold_P = 50
    threshold_O = 120
    threshold_F = 300 
    df1_1.loc[file_name] = main1(file_name, threshold_P, threshold_O, threshold_F)

ValueError: cannot set a row with mismatched columns

In [14]:
df1_2 = pd.DataFrame(data=None, index=None, columns= ["FKBP8_on_mito","r_FKBP8_on_mito","FKBP8_out_mito"], dtype=None, copy=False)
for i in [1,2,4,5]:
    file_path = r"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 " + str(i) + " - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_"
    threshold_P = 40
    threshold_O = 120
    threshold_F = 400
    df1_2.loc[file_path] = main1(file_path,threshold_P,threshold_O,threshold_F)

In [15]:
df = pd.concat([df1_1, df1_2])
df

,FKBP8_on_mito,r_FKBP8_on_mito,FKBP8_out_mito
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 1 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.831671,0.370201,0.168329
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 6 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.859028,0.329290,0.140972
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 7 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.844640,0.480041,0.155360
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 9 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.837610,0.369249,0.162390
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221122 Nagao/Analysis/Split images/221122Exp268 KI endoFKBP8 PDZD8-Halo Tom20 10 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.873158,0.342737,0.126842
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 1 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.891941,0.385081,0.108059
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 2 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.900994,0.445177,0.099006
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 4 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.961225,0.522895,0.038775
"/Users/sa/Documents/SA/Hirabayashi Lab/Experiment/Exp_Data/Imaging/221126 Nagao/Analysis/Split images/221126Exp268-2 KI endoFKBP8 PDZD8-Halo Tom20 5 - Deconvolved 20 iterations, Type Richardson-Lucy.nd2 cropped_",0.939781,0.291846,0.060219


In [210]:
df1.to_csv("FKBP8_Manders.csv")